# 《Reinforcement Learning, Second Edition 2018》笔记

![](https://github.com/applenob/rl_learn/raw/master/res/rl_book.jpg)

## 目录

- [0.书本介绍](#0.书本介绍)
- [1. Introduction](#1.-Introduction)
- [2. Multi-armed Bandits](#2.-Multi-armed-Bandits)
- [3. Finite Markov DecisionProcesses](#3.-Finite-Markov-DecisionProcesses)
- [4. Dynamic Programming](#4.-Dynamic-Programming)
- [5. Monte Carlo Methods](#5.-Monte-Carlo-Methods)
- [6. Temporal-Difference Learning](#6.-Temporal-Difference-Learning)
- [7. n-step Bootstrapping](#7.-n-step-Bootstrapping)
- [8. Planning and Learning with Tabular Methods](#8.-Planning-and-Learning-with-Tabular-Methods)
- [9. On-policy Prediction with Approximation](#9.-On-policy-Prediction-with-Approximation)
- [10. On-policy Control with Approximation](#10.-On-policy-Control-with-Approximation)
- [11. Off-policy Methods with Approximation](#11.-Off-policy-Methods-with-Approximation)
- [12. Eligibility Traces](#12.-Eligibility-Traces)
- [13. Policy Gradient Methods](#13.-Policy-Gradient-Methods)
- [14. Psychology](#14.-Psychology)
- [15. Neuroscience](#15.-Neuroscience)
- [16. Applications and Case Studies](#16.-Applications-and-Case-Studies)
- [17. Frontiers](#17.-Frontiers)


## 0.书本介绍

[Reinforcement Learning](https://book.douban.com/subject/2866455/)

作者: Richard S. Sutton / Andrew G. Barto 

几乎是最权威的强化学习教材。

### 重要的符号表示
- $q_∗(a)$：动作a对应的真实价值（价值期望），true value (expected reward) of action a
- $Q_t(a)$：动作a在t时刻的估计价值，estimate at time t of $q_∗(a)$
- $N_t(a)$：在t时刻之前，动作a已经被选择的次数，number of times action a has been selected up prior to time t
- $H_t(a)$：在t时刻选择动作a的偏好程度，learned preference for selecting action a at time t
- $π_t(a)$：在t时刻选择动作a的概率，probability of selecting action a at time t
- $R_t$：给定策略$π_t$，在t时刻的期望奖励，estimate at time t of the expected reward given$π_t$


## 1. Introduction

### 什么是强化学习
强化学习是“学习该如何去做”（learning what to do），即学习如何从**某个状态映射到某个行为，来最大化某个数值的奖励信号**。

### 强化学习的特征
强化学习两个最重要的特征：
- **试错（trial-and-error search ）**：agent需要不断尝试，通过reward的反馈学习策略。
- **延迟奖励（delayed reward） **：某个时刻的action可能会对后面时刻的reward有影响（深远影响）。

### Exploit vs Explore
**exploit** 代表利用已有的信息去获得最大的奖励，**explore** 代表去探索一些没有尝试过的行为，去获得可能的更大的奖励。

### 强化学习的几个要素
- **policy**: 状态到行为的映射，定义agent在某个状态下应该如何采取行为，state $\rightarrow$ action。
- **reward function**: 在某个状态下，agent能够收到的**即时反馈**。
- **value function**: 衡量在某个状态下，能够获得的**长期反馈**。
- **modle (of the environment，可选的)**: 模型用来模拟环境的行为，**给定一个状态和动作，模型可以预测下一个状态和奖励**。
   
### RL vs Evolutionary Methods
- Evolutionary Methods（遗传算法，具体可以回顾[之前的博客](https://applenob.github.io/ga.html)），直接在policy空间中搜索，并计算最后的得分。通过一代代的进化来找寻最优policy。
- 遗传算法忽略了policy实际上是state到action的映射，它不关注agent和环境的互动，只看最终结果。

### 局限性
强化学习非常依赖**状态（state）**的概念。state既是策略函数和价值函数的输入，又是环境模型（model）的输入和输出。

### Tic-Tac-Toe（井字棋）
- ![](https://github.com/applenob/rl_learn/raw/master/res/ttt.png)
- 一个简单的应用强化学习的例子。
- 定义policy：任何一种局面下，该如何落子。
- 遗传算法解法：试很多种policy，找到最终胜利的几种，然后结合，更新。
- 强化学习解法：
    - 1.建立一张表格，state_num × 1，代表每个state下，获胜的概率，这个表格就是所谓的**value function**，即状态到价值的映射。
    - 2.跟对手下很多局。每次落子的时候，依据是在某个state下，选择所有可能的后继state中，获胜概率最大的（value最大的）。这种方法即贪婪法（Exploit）。偶尔我们也随机选择一些其他的state（Explore）。
    - 3.“back up”后继state的v到当前state上。$V(s)\leftarrow V(s)+\alpha[V(s')-V(s)]$，这就是所谓的**差分学习（temporal-difference learning）**，这么叫是因为$V(s')-V(s)$是两个时间点上的两次估计的差。

## 2. Multi-armed Bandits

### k-armed Bandit Problem，K臂老虎机
- 问题描述：重复地面临有k中选择的情况，每一次选择之后，都会收到一个reward。这个reward服从一个跟你的选择相关的分布。你的目标是，在选择t次后，找到最大的期望reward。
- 为什么叫K臂老虎机：有K个单臂老虎机，每个时间节点，你可以选择K个中任意一个老虎机选择按下它的臂，然后获得一个奖励。目标自然是多次选择之后的累计奖励最大。
- 形式化定义：假设时刻$t$的动作是$A_t$，reward是$R_t$，定义价值函数：$q_*(a) = E[R_t|A_t=a]$。
- 特点：每个时刻下，$q_*(a)$是固定的，stationary。
- 如果我们知道每个动作a对应的价值q，那么这个问题对我们来说就已经解决了，即，我们选择q最大的那个a。
- 然而，我们并不知道实际上的q是什么，只能估计$Q_t(a)\approx q_*(a)$。
- 在每个时间节点上，如果总是选择对应的估计Q最大的a，这被称为**greedy**。
- **$\epsilon$-greedy**：*：$1-\epsilon$的概率选择Q最大的a，*$\epsilon$的概率选择其他a。
- 估计$Q_t(a) = \frac{在时刻t之前，所有采用动作a获得的奖励之和}{在时刻t之前，采用动作a的次数} = \frac{\sum^{t-1}_{i=1}R_i\cdot\mathbf{1}_{A_i=a}}{\sum^{t-1}_{i=1}\mathbf{1}_{A_i=a}}$，被称为sample-average。

### 10-armed Testbed
- 设置一个10臂老虎机作模拟。
- 真实的价值函数$q_*(a)$服从标准高斯分布（均值是0，方差是1）。
- t时刻的奖励$R_t$服从高斯分布，均值是$q_*(A_t)$，方差是1。
- 可以规定每次实验做1000次选择，称为一个run。
- 一共跑了2000个独立的run。
- 实验结果（**$\epsilon$-greedy**的优越性）：
- ![](https://github.com/applenob/rl_learn/raw/master/res/average_reward.png)

### 增量式的估计
- 上面的sample-average即：$Q_n = \frac{R_1+R_2+...+R_{n-1}}{n-1}$
- 改写成增量式的形式：$Q_{n+1} = Q_n + \frac{1}{n}[R_n-Q_n]$
- 即：新估计 = 老估计 + 步长 × [奖励 - 老估计]
- ![](https://github.com/applenob/rl_learn/raw/master/res/bandit_algo.png)

### Optimistic Initial Values
- 设置初始的$Q_1(a)$为一些较大的值。
- 这样会鼓励explore。
- 对于nonstationary的问题不适用。
- ![](https://github.com/applenob/rl_learn/raw/master/res/optimistic.png)

### Upper-Confidence-Bound Action Selection（UCB）
- $A_t = \underset{a}{argmax}[Q_t(a)+c\sqrt{\frac{log\;t}{N_t(a)}}]$
- 其中，$N_t(a)$代表动作a在t之前出现的次数，根号项衡量动作a的不确定性，如果某个动作已经使用了很多次，则倾向使用使用次数少的，这样达到explore的效果。
- ![](https://github.com/applenob/rl_learn/raw/master/res/ucb.png)

### Gradient Bandit Algorithms
- 使用一个数值表示对某个动作的**偏好**：$H_t(a)$
- $Pr\{A_t=a\}=\frac{e^{H_t(a)}}{\sum_{b=1}^k{e^{H_t(b)}}}=\pi_t(a)$
- 更新规则（不必细究，学到policy gradient自然明白）：
    - $H_{t+1}(A_t) = H_t(A_t) + \alpha(R_t-\overline R_t)(1-\pi _t(A_t))$
    - $\forall a \neq A_t$
- ![](https://github.com/applenob/rl_learn/raw/master/res/gradient_bandit.png)

### 各种方法对比
- ![](https://github.com/applenob/rl_learn/raw/master/res/bandit_methods.png)

## 3. Finite Markov DecisionProcesses

### Agent和Environment的交互
- 学习者和决策者称为agent。
- agent交互的对象，外部环境，称为Environment。
- 在时刻t，agent的所处的环境用状态：$S_t \in S$表示，$S$是可能的状态集。假设agent采用了动作$A_t\in A(S_t)$，$A(S_t)$代表在状态$S_t$下可能的动作集。
- 到了下一个时刻t+1，agent收到了一个奖励：$R_{t+1} \in R$，并且发现自己处在一个新的state中：$S_{t+1}$。
- ![](https://github.com/applenob/rl_learn/raw/master/res/agent_env.png)

### 什么是有限
有限MDP中的有限意味着：状态空间S、动作空间A和奖励空间R都是离散且有限的。

### 目标和奖励的区别
- 每个时间节点上，agent都会收到一个奖励的数值：$R_t$。
- 但是，agent的目标应该是：所有时间结点上的奖励的和的最大化。
- 即：$$G_t = R_{t+1} +  R_{t+2} + ... + R_T$$

### 什么是Episode
一系列的agent和environment交互序列。每个episode之间相互不影响，且都是有一个相同的终止状态（terminate state）。

### Continuing Tasks
不同于Episode类的任务，这类任务没有终止状态，也就是说$T = \infty$。因此，如果考虑这类任务，那么$G_t$将是无穷大，于是引入discounting。

### Episode和Continuing Tasks的统一规范
在Episode的尾部加入吸收状态（absorbing state），在此状态下，奖励永远是0，且下一个状态永远是当前状态。

这样收益可以统一使用下面的Expected Discounted Return表示。

### Expected Discounted Return
- 回报：$G_t = R_{t+1} +  \gamma R_{t+2} + ... = \sum_{k=0}^{\infty} \gamma ^k R_{t+k+1}$
- $\gamma$是参数，且$0\leq \gamma \leq 1$，被称为**discount rate**。
- 含义：一个奖励，如果是在k个时间节点以后收到，那么对于当前来说，它的价值是即时奖励的$\gamma^{k-1}$倍。
- 从$G_t$的定义，很容易获得递推式：$G_t = R_{t+1} + \gamma G_{t+1}$

### 马尔科夫性质（Markov property）
- 核心思想：**当前state继承了所有的环境历史信息。**
- $$Pr\{S_{t+1}=s', R_{t+1}=r|S_0,A_0,R_1,...,S_{t-1},A_{t_1},R_t,S_t,A_t\} = Pr\{S_{t+1}=s', R_{t+1}=r|S_t,A_t\}$$
- 即便state是非马尔科夫的，我们也希望近似到马尔科夫。

### Markov Dicision Processes（MDP）
- **满足马尔科夫性质的强化学习任务称为MDP。**
- 如果state和action空间有限，则称为有限MDP（涵盖了现代强化学习90%的问题）。
- 用$p(s',r|s,a)$表示$Pr\{S_{t+1}={s}', R_{t+1}=r|S_t,A_t\}$，这个条件概率是描绘了整个MDP的动态（Dynamics）。
- state-action期望奖励：$$r(s,a) = \mathbb{E}[R_{t+1}|S_t=s,A_t=a]=\sum_{r\in R}r\sum_{s'\in S}p(s', r|s,a)$$
- 状态转移概率：$$p(s'|s,a) = Pr\{S_{t+1}=s'|S_t=s, A_t=a\}=\sum_{r\in R}p(s', r| s, a)$$
- state-action-nextstate期望奖励：$$r(s, a, s') = \mathbb{E}[R_{t+1}|S_t=s,A_t=a, S_{t+1}=s'] = \sum_{r\in R}r\frac{p(s',r|s,a)}{p(s'|s,a)}$$

### 价值函数
- 关于策略$\pi$的state-value函数：$$v_{\pi}(s) = \mathbb{E}_{\pi}[G_t|S_t=s]=\mathbb{E}_{\pi}[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s]$$
- 即，**在使用策略$\pi$的前提下，衡量处于某个state有多好**。
- 关于策略$\pi$的action-value函数：$$q_{\pi}(a,s) = \mathbb{E}_{\pi}[G_t|S_t=s,A_t=a] = \mathbb{E}_{\pi}[\sum_{k=0}^{\infty}\gamma^kR_{t+k+1}|S_t=s,A_t=a]$$
- 即，在使用策略$\pi$的前提下，衡量处于某个state下，执行某个action有多好。

### Bellman Euqation
- **Bellman Expectation Euqation for $v_{\pi}$**：$$\begin{align} v_{\pi}(s) = \sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_{\pi}(s')]\;\;\forall s \in S \end{align}$$
- 理解：
    - 1.方括号中是根据后继状态的价值重新估计的价值函数，再在动作空间、后继状态空间和动作空间用相应的概率做加权求和。
    - 2.**表达的是某个状态的价值和其后继状态的价值之间的关系。**
- **backup**：是强化学习方法的核心，以时序意义上的回退，用下一个时刻的值去评估当前时刻的值。
- **Bellman Expectation Euqation for $q_{\pi}$**：$$q_{\pi}(s,a) = \sum_{s'}p(s',r|s,a)[r+\gamma \sum_{a'}q(s',a')]$$    
- ![](https://github.com/applenob/rl_learn/raw/master/res/backup.png)
- 推导：
$$\begin{align}
  v_\pi(s) & = \mathbb{E}_\pi\left[G_t \mid S_t = s\right] \\
  & = \mathbb{E}_\pi\left[R_{t+1} + \gamma G_{t+1} \mid S_t = s\right] \\
  & = \sum_a \pi(a \mid s) \sum_{s', r} p(s', r \mid s, a) \left[r + \gamma v_\pi(s')\right].
\end{align}$$
$$\begin{align}
q_{\pi}(s,a) & = \mathbb{E}_\pi\left[G_t \mid S_t = s, A_t = a\right] \\
& = \mathbb{E}_\pi\left[R_{t+1} + \gamma G_{t+1} \mid S_t = s, A_t = a\right] \\
& = \sum_{s',r}p(s',r|s,a)[r+\gamma \sum_{a'}q(s',a')]
\end{align}$$
- 参考资料：https://joshgreaves.com/reinforcement-learning/understanding-rl-the-bellman-equations/

### 最优化价值函数
- $v_*(s) = \underset{\pi}{max}v_{\pi}(s)$
- $q_*(s,a) = \underset{\pi}{max}q_{\pi}(s,a)$
- **Bellman Optimality Euqation for $v_*(s)$**：$$v_*(s)=\underset{a\in A(s)}{max}\sum_{s',r}p(s',r|s,a)[r+\gamma v_*(s')]$$
- **Bellman Optimality Euqation for $q_*(s,a)$**：$$q_*(s,a)=\sum_{s',r}p(s',r|s,a)[r+\gamma \underset{a'}{max}q_*(s', a')]$$
- ![](https://github.com/applenob/rl_learn/raw/master/res/backup_opt.png)


## 4. Dynamic Programming

### DP方法简介
- 由于其大量的计算损耗，已经不实用，但理论上非常重要。
- 本书后续的所有方法可以看做想要取得和DP类似的效果；只不过是减少了计算或者假设没有完美的环境模型。
- 假设解决的问题是有限的MDP，即给定动作a，状态s，和奖励r，可以通过$p(s',r|s,a)$描述动态变化。

### Policy Evaluation
- **评估一个策略的好坏。**
- 策略评估，**计算某个policy对应的价值函数**，也被称为prediction problem。
- 更新方法：使用上一章讲的**Bellman Expectation Euqation for $v_{\pi}$**：$$v_{\pi}(s) = \sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)[r+\gamma v_{\pi}(s')]\;\;\forall s \in S$$
- ![](https://github.com/applenob/rl_learn/raw/master/res/iterative_policy_evaluation.png)

### Policy Improvement
- 在当前的策略和相应的价值函数的基础上，使用价值函数贪婪地更新当前策略的过程。
- policy improvement theorem：对所有的$s \in S$，有$q_{\pi}(s, \pi'(s)) \geq v_{\pi}(s)$，则$v_{\pi'}\geq v_{\pi}(s)$，即策略$\pi'$优于策略$\pi$。
- greedy policy：$$\pi'(s)=\underset{a}{argmax}q_{\pi}(s,a)=\underset{a}{argmax}\sum_{s', r}p(s', r|s,a)[r+\gamma v_{\pi}(s')]$$

### Policy Iteration
- ![](https://github.com/applenob/rl_learn/raw/master/res/policy_iteration_2.png)
- 其中，$E$代表策略的evaluation，$I$代表策略的improvement。简单地说，Policy Iteration就是不断地评估策略然后改进策略。
- ![](https://github.com/applenob/rl_learn/raw/master/res/policy_iteration.png)

### Value Iteration
- Policy Iteration的缺点：每一次迭代都要评估策略，而每一次策略评估本身都是一个迭代过程。
- $$v_{k+1}(s)=\underset{a}{max} E[R_{t+1}+\gamma v_k(S_{t+1})|S_t=s, A_t=a]\\=\underset{a}{max}\sum_{s',r}p(s',r|s,a)[r+\gamma v_k(s')]$$
实际上就是**Bellman Optimality Euqation for $v_*(s)$**。
- ![](https://github.com/applenob/rl_learn/raw/master/res/value_iteration.png)
- Value Iteration简单地说就是每次评估价值的时候直接用可能的用最优价值函数更新价值函数（这样的每一步不涉及策略本身）；在确定已经获得比较准确的价值评估之后，再一次性确定策略。

## 5. Monte Carlo Methods

### 蒙特卡洛方法简介
- 使用蒙特卡洛方法不需要像DP一样，对环境要有完整的知识，而是通过**经验**去学习。所谓经验就是对状态、动作、奖励的采样（sample sequence）。
- 用sample的均值去近似期望。
- 使用蒙特卡洛通常需要完整的episode，因此蒙特卡洛的更新方式更像是**episode-by-episode**，而不是像DP的step-by-step。
- 优点：
    - 1.可以从实际经验中学习；
    - 2.可以从模拟的经验中学习；
    - 3.可以直接从感兴趣的state开始采样episode。

### 蒙特卡洛预测（评估）
- 在一个episode中状态s可能出现多次，每一次出现称为一次对状态s的**访问（visit）**。
- **first-visit MC method**:只是用每个episode中第一次对状态s的访问评估状态s的价值函数。
- **every-visit MC method**:用每个episode中每次对状态s的访问评估状态s的价值函数。
- ![](https://github.com/applenob/rl_learn/raw/master/res/fv_mc_est.png)
- $V(s)\leftarrow average(Return(s))$

### 蒙特卡洛评估动作价值函数（Action Values）

- **注意**：如果我们的问题中，没有对环境建模，那么单纯评估状态价值函数是不够的。我们必须要评估动作价值函数。
- **主体思想**：从评估state到评估state-action对。
- **可能存在的问题**：某些state-action对可能不会被访问（稀疏性问题）。

### 蒙特卡洛控制
- **控制（control）**的目的是找到最优策略。
- ![](https://github.com/applenob/rl_learn/raw/master/res/mc_policy_iter.png)
- 其中，$E$代表策略的evaluation，$I$代表策略的improvement。

### Monte Carlo Exploring Starts
- **Exploring Starts**：所有的state-action对都有可能被选为episode的开始（start）。
- ![](https://github.com/applenob/rl_learn/raw/master/res/mces.png)

### 不使用Exploring Starts
- 如何才能不使用Exploring Starts？保证所有被选择的动作被持续地选择。
- 使用on-policy和off-policy。

### on-policy vs off-policy
- on-policy只有一套policy，更简单，是首选。
- off-policy使用两套policy，更复杂、更难收敛；但也更通用、更强大。
- on-policy和off-policy本质依然是Exploit vs Explore的权衡。

### on-policy
- 去评估和提高生成episode时采用的policy。**全过程只有一种策略**，MC ES属于on-policy。

### off-policy
- 所有的MC控制方法都面临一个**困境**：它们都想找到一个最优的策略，但却必须采用非最优的策略去尽可能多地探索（explore）数据。
- 直接使用**两套策略**：采样用的policy称为`behavior policy`，最终的目标policy：`target policy`。这就是off-policy。
- 假设目标策略是$\pi$，行为策略是$b$，那么对于所有的$\pi(a|s)>0$必然有$b(a|s)>0$，这称为“覆盖”（coverage）。一个常见的例子是：行为策略使用价值函数的greedy policy，而行为策略使用ε-greedy policy。

### 重要性采样（importance sampling）
- 几乎所有的off-policy都使用**重要性采样（importance sampling）**。
- 为什么要使用重要性采样？我们希望在使用目标策略$\pi$的情况下用均值估计价值的期望，但我们获得的是在使用行为策略$b$的情况下的均值，也就是：$\mathbb{E}[G_t \mid S_t =s] = v_b(s)$。这二者是有差距的。因此我们希望使用重要性采样去纠正。
- 给定初始状态$S_t$，后续的状态-动作轨迹在使用策略$\pi$的情况下的概率为：
$$Pr\{At,S_{t+1}, A_{t+1}, ... S_T \mid S_t, A_{t:T −1} \sim \pi\}=\prod_{k=t}^{T-1}\pi(A_k\mid S_k)p(S_{k+1}\mid S_k, A_k)$$
- 引入**重要性采样比例（the importancesampling ratio）**：
$$\rho_{t:T −1}=\frac{\prod_{k=t}^{T-1}\pi(A_k\mid S_k)p(S_{k+1}\mid S_k, A_k)}{\prod_{k=t}^{T-1}b(A_k\mid S_k)p(S_{k+1}\mid S_k, A_k)}=\prod_{k=t}^{T-1}\frac{\pi(A_k\mid S_k)}{b(A_k\mid S_k)}$$
上面这个式子正好巧妙地把MDP中未知的状态转移概率约掉。
- 于是return的期望又可以得到校正：$\mathbb{E}[\rho_{t:T−1}G_t \mid S_t =s] = v_{\pi}(s)$
- odinary importance sampling：
$$V(s) = \frac{\sum_{t\in J(s)} \rho_{t:T (t)-1}Gt}{\mid J(s)\mid} $$
- weighted importance sampling：
$$V(s) = \frac{\sum_{t\in J(s)} \rho_{t:T (t)-1}Gt}{\sum_{t\in J(s)} \rho_{t:T (t)-1}} $$
- odinary importance sampling vs. weighted importance sampling:
    - odinary importance sampling：无偏差，但方差没有保证。
    - weighted importance sampling：有偏差，方差有上限。
- 评估：
- ![](https://github.com/applenob/rl_learn/raw/master/res/off_policy_mc_prediction.png)
- 上面的评估使用了采样权重增量式的方法。
- 控制：
- ![](https://github.com/applenob/rl_learn/raw/master/res/off_policy_mc_control.png)



## 6. Temporal-Difference Learning


### 时序差分（Temporal-Difference）简介
- 时序差分是强化学习的核心观点。
- 时序差分是DP和MC方法的结合。
- 时序差分不需要像MC一样，要等一个完整的序列结束；相反，每经历一步，都会更新价值函数。
- TD往往比MC高效
- 所谓差分就是下一个时刻的估计和当前时刻的估计的差。

### 什么是stationary？
- stationary：环境不随时间变化而变化；
- non-stationary：环境会随时间变化而变化。

### TD(0)
- $V(S_t)\leftarrow V(S_t)+\alpha[R_{t+1}+\gamma V(S_{t+1})-V(S_t)]$
- 因为直接使用现有的估计取更新估计，因此这种方法被称为**自举（bootstrap）**。
- ![](https://github.com/applenob/rl_learn/raw/master/res/td0_est.png)
- **TD error**：$\delta_t = R_{t+1}+\gamma V(S_{t+1})-V(S_t)$
- ![](https://github.com/applenob/rl_learn/raw/master/res/td_0.png)

### Sarsa
- 一种on-policy的TD控制。
- $Q(S_t,A_t)\leftarrow Q(S_t,A_t)+\alpha[R_{t+1}+\gamma Q(S_{t+1},A_{t+1})-Q(S_t,A_t)]$
- ![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_est.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/sarsa_backup.png)

### Q-learning
- 一种off-policy的TD控制。
- 早期强化学习的一个突破。
- $Q(S_t,A_t)\leftarrow Q(S_t,A_t)+\alpha[R_{t+1}+\gamma \underset{a}{max}Q(S_{t+1},a)-Q(S_t,A_t)]$
- ![](https://github.com/applenob/rl_learn/raw/master/res/q_learn.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/q_learn_backup.png)

### Expected Sarsa
- 一种off-policy的TD控制。
- $Q(S_t,A_t)\leftarrow Q(S_t,A_t) + \alpha[R_{t+1} + \gamma\sum_a\pi(a|S_{t+1})Q(S_{t+1}, a)-Q(S_t,A_t)]$

### Double Learning
- 解决Q-learning的**最大化偏差（maximization bias）**问题
- 2011年提出。
- ![](https://github.com/applenob/rl_learn/raw/master/res/double_q_learn.png)

## 7. n-step Bootstrapping

### n-step Bootstrapping简介
- 是MC方法和一步TD方法的结合。
- 是资格痕迹（eligibility traces）的基础先验知识。

### n-step TD Prediction
- MC对应的回报：$G_t = R_{t+1}+\gamma R_{t+2}+...+\gamma^{T-t-1}R_T$
- 一步TD对应的回报：$G_{t:t+1} = R_{t+1} + \gamma V_t(S_{t+1})$
- n-step的回报：$G_{t:t+n} = R_{t+1} + \gamma R_{t+2}+...+\gamma^{n-1}R_{t+n}+\gamma^nV_{t+n+1}(S_{t+n})$
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-pred.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-est.png)

### n-step Sarsa
- n-step Sarsa: 
- 回报：$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \gamma^{n-1} R_{t+n} + \gamma^nQ_{t+n-1}(S_{t+n}, A_{t+n})$
- 更新动作价值函数：$Q_{t+n}(S_t, A_t) = Q_{t+n-1}(S_t, A_t) + \alpha[G_{t:t+n} - Q_{t+n-1}(S_t,A_t)]$
- n-step Expected Sarsa:
- 回报：$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \gamma^{n-1} R_{t+n} + \gamma^n \bar V_{t+n-1}(S_{t+n})$
- 更新价值函数：$\bar V_t(s) = \sum_a \pi(a|s)Q_t(s,a)$
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-sarsa.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-sarsa-est.png)

### n-step off-policy with Importance Sampling
- 更新动作价值函数：$Q_{t+n}(S_t， A_t) = Q_{t+n-1}(S_t, A_t) + \alpha \rho_{t:t+n-1}[G_{t:t+n} - Q_{t+n-1}(S_t, A_t)]$
- 其中，重要性采样比例：$\rho_{t:h} = \prod_{k=t}^{min(h,T-1)}\frac{\pi(A_k|S_k)}{b(A_k, S_k)}$
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-off.png)

### n-step Tree Backup Algorithm
- 不需要重要性采样。
- 使用所有**叶子节点**的动作价值函数去更新动作价值函数。
- 回报：$G_{t:t+n} = R_{t+1} + \gamma \sum_{a\neq A_{t+1}}\pi(a|S_{t+1})Q_{t+n-1}(S_{t+1}, a) + \gamma\pi(A_{t+1}|S_{t+1})G_{t+1:t+n}$
- ![](https://github.com/applenob/rl_learn/raw/master/res/3-step-tree.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-tree.png)

### n-step $Q(\sigma)$
- $\sigma$代表是否使用全采样。
- 回报：$G_{t:h} = R_{t+1} + \gamma(\sigma_{t+1}\rho_{t+1}+(1-\sigma_{t+1})\pi(A_{t+1}|S_{t+1}))(G_{t+1:h}-Q_{h-1}(S_{t+1}, A_{t+1})) + \gamma \bar V_{h-1}(S_{t+1})$
- ![](https://github.com/applenob/rl_learn/raw/master/res/4-step-q.png)
- ![](https://github.com/applenob/rl_learn/raw/master/res/n-step-q.png)

## 8. Planning and Learning with Tabular Methods

### Tabular Dyna-Q
- ![](https://github.com/applenob/rl_learn/raw/master/res/tab_dyna_q.png)

## 9. On-policy Prediction with Approximation

## 10. On-policy Control with Approximation

## 11. Off-policy Methods with Approximation

## 12. Eligibility Traces

## 13. Policy Gradient Methods

## 14. Psychology

## 15. Neuroscience

## 16. Applications and Case Studies

## 17. Frontiers

In [1]:
print "done"

done
